## 🎓 Lab Overview

This hands-on session demonstrates **object detection for Earth observation** using transfer learning. You'll fine-tune a pre-trained object detection model to detect buildings and informal settlements in Metro Manila from satellite imagery.

**Case Study:** Metro Manila Building Detection  
**Duration:** 2.5 hours  
**Platform:** Google Colab with GPU  
**Dataset:** Synthetic Sentinel-2-like imagery (for rapid learning)

---

## Learning Objectives

By the end of this lab, you will be able to:

1. ✅ **Understand** transfer learning for object detection in EO applications
2. ✅ **Load and configure** pre-trained models from TensorFlow Hub
3. ✅ **Prepare** satellite imagery and annotations for object detection
4. ✅ **Implement** Non-Maximum Suppression (NMS) for post-processing
5. ✅ **Evaluate** model performance using mAP (mean Average Precision)
6. ✅ **Compare** different detection architectures (SSD, EfficientDet)
7. ✅ **Visualize** detected bounding boxes and analyze predictions
8. ✅ **Export** models for operational deployment

---

## Lab Structure

| Step | Activity | Duration |
|------|----------|----------|
| **1** | Environment Setup & GPU Check | 5 min |
| **2** | Dataset Generation & Exploration | 15 min |
| **3** | Data Format & Annotation | 15 min |
| **4** | Load Pre-trained Models | 15 min |
| **5** | Architecture Comparison | 20 min |
| **6** | Non-Maximum Suppression (NMS) | 20 min |
| **7** | Evaluation with mAP | 25 min |
| **8** | Advanced Visualization | 15 min |
| **9** | Export & Deployment | 15 min |
| **10** | Troubleshooting & Best Practices | 10 min |

**Total:** ~150 minutes

---

# Step 1: Environment Setup (5 minutes)

First, let's install required packages and check GPU availability.

In [ ]:
# Install required packages
!pip install -q tensorflow-hub pycocotools tensorflow-addons
print("✅ Packages installed!")

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import seaborn as sns
import os
import json
import random
from PIL import Image
from collections import defaultdict

# TensorFlow and related
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ NumPy version: {np.__version__}")

### Check GPU Availability

Object detection benefits significantly from GPU acceleration.

In [ ]:
# Check for GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"\n✓ GPU(s) Available: {len(gpus)}")
    for gpu in gpus:
        print(f"  - {gpu.name}")
    
    # Enable memory growth
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("\n✓ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("\n⚠️  No GPU found - inference will use CPU (slower)")
    print("   Consider using Google Colab with GPU runtime")

print("\n✓ Environment ready!")

---

# Step 2: Dataset Generation & Exploration (15 minutes)

For this lab, we'll generate **synthetic Sentinel-2-like urban imagery** with building annotations. This allows immediate execution while teaching the complete workflow.

## Philippine Context: Metro Manila Urban Monitoring

::: {.callout-note}
## Why Building Detection Matters in Metro Manila

**Location:** National Capital Region (NCR) - 17 cities, 16.7M population  
**Challenge:** Rapid urbanization and informal settlement growth

**Applications:**
- **Disaster Risk Reduction:** Identify vulnerable settlements in flood zones
- **Urban Planning:** Monitor informal settlements and infrastructure
- **Population Estimation:** Building counts for demographic analysis  
- **Change Detection:** Track urban expansion over time
- **Resource Allocation:** Target social services and infrastructure development

**Data Source:** Sentinel-2 Multispectral Imagery
- 10m resolution (RGB bands)
- 5-day revisit frequency
- Free and open access
:::

### Generate Synthetic Dataset

We'll create realistic urban scenes with buildings of varying sizes:

In [ ]:
def generate_urban_imagery(n_samples=100, img_size=320, seed=42):
    """
    Generate synthetic Sentinel-2-like urban imagery with building annotations
    
    Args:
        n_samples: Number of images to generate
        img_size: Image dimensions (default 320x320)
        seed: Random seed for reproducibility
    
    Returns:
        images: List of RGB images (normalized to [0, 1])
        boxes_list: List of bounding boxes per image [y1, x1, y2, x2] normalized
        labels_list: List of class labels per image (all 1 for 'building')
    """
    np.random.seed(seed)
    print(f"Generating {n_samples} synthetic urban scenes...")
    
    images = []
    boxes_list = []
    labels_list = []
    
    for i in range(n_samples):
        # Create base urban scene (gray/brown tones)
        base_color = np.random.randint(60, 100, 3)
        img = np.ones((img_size, img_size, 3), dtype=np.uint8) * base_color
        
        # Add texture (simulates roads, vegetation patches)
        noise = np.random.randint(-15, 15, (img_size, img_size, 3))
        img = img + noise
        img = np.clip(img, 0, 255).astype(np.uint8)
        
        # Add random vegetation patches (darker green areas)
        n_veg_patches = np.random.randint(1, 4)
        for _ in range(n_veg_patches):
            veg_x = np.random.randint(0, img_size-30)
            veg_y = np.random.randint(0, img_size-30)
            veg_size = np.random.randint(15, 40)
            veg_color = np.array([50, 80, 50])  # Green-ish
            img[veg_y:veg_y+veg_size, veg_x:veg_x+veg_size] = veg_color
        
        # Add buildings (3-10 per image)
        n_buildings = np.random.randint(3, 11)
        boxes = []
        labels = []
        
        for _ in range(n_buildings):
            # Random building location
            x = np.random.randint(10, img_size-60)
            y = np.random.randint(10, img_size-60)
            
            # Random building size (small, medium, large)
            size_type = np.random.choice(['small', 'medium', 'large'], p=[0.5, 0.3, 0.2])
            if size_type == 'small':
                w = np.random.randint(12, 25)
                h = np.random.randint(12, 25)
            elif size_type == 'medium':
                w = np.random.randint(25, 45)
                h = np.random.randint(25, 45)
            else:  # large
                w = np.random.randint(45, 70)
                h = np.random.randint(45, 70)
            
            # Check bounds
            if x + w >= img_size or y + h >= img_size:
                continue
            
            # Draw building (bright rectangle - concrete/metal roofs)
            building_color = np.random.randint(150, 230, 3)
            img[y:y+h, x:x+w] = building_color
            
            # Add some building detail (darker edges for realism)
            edge_width = 2
            img[y:y+edge_width, x:x+w] = building_color * 0.7  # Top edge
            img[y:y+h, x:x+edge_width] = building_color * 0.7  # Left edge
            
            # Normalized bbox [y1, x1, y2, x2] - COCO format
            box = [y/img_size, x/img_size, (y+h)/img_size, (x+w)/img_size]
            boxes.append(box)
            labels.append(1)  # Class 1 = building
        
        # Normalize image to [0, 1]
        img_normalized = img.astype(np.float32) / 255.0
        
        images.append(img_normalized)
        boxes_list.append(np.array(boxes, dtype=np.float32))
        labels_list.append(np.array(labels, dtype=np.int32))
        
        if (i+1) % 25 == 0:
            print(f"  Generated {i+1}/{n_samples} images")
    
    print(f"\n✅ Dataset generated successfully!")
    print(f"   Total images: {len(images)}")
    print(f"   Image shape: {images[0].shape}")
    print(f"   Buildings per image: {np.mean([len(b) for b in boxes_list]):.1f} (avg)")
    
    return images, boxes_list, labels_list

# Generate dataset
all_images, all_boxes, all_labels = generate_urban_imagery(
    n_samples=100,
    img_size=320,
    seed=42
)

### Split Dataset

Split into train (70%), validation (15%), and test (15%) sets:

In [ ]:
# Split dataset: 70/15/15
n_train = int(0.70 * len(all_images))
n_val = int(0.15 * len(all_images))

train_images = all_images[:n_train]
train_boxes = all_boxes[:n_train]
train_labels = all_labels[:n_train]

val_images = all_images[n_train:n_train+n_val]
val_boxes = all_boxes[n_train:n_train+n_val]
val_labels = all_labels[n_train:n_train+n_val]

test_images = all_images[n_train+n_val:]
test_boxes = all_boxes[n_train+n_val:]
test_labels = all_labels[n_train+n_val:]

print(f"Dataset Split:")
print(f"  Train: {len(train_images)} images, {sum(len(b) for b in train_boxes)} buildings")
print(f"  Val:   {len(val_images)} images, {sum(len(b) for b in val_boxes)} buildings")
print(f"  Test:  {len(test_images)} images, {sum(len(b) for b in test_boxes)} buildings")

### Visualize Sample Images

Let's examine the synthetic urban scenes with building annotations:

In [ ]:
def visualize_annotated_images(images, boxes_list, labels_list, n_samples=6, title="Annotated Images"):
    """
    Visualize images with bounding box annotations
    """
    fig, axes = plt.subplots(2, 3, figsize=(16, 11))
    axes = axes.ravel()
    
    for i in range(min(n_samples, len(images))):
        ax = axes[i]
        
        # Display image
        ax.imshow(images[i])
        
        # Draw bounding boxes
        img_h, img_w = 320, 320
        for box, label in zip(boxes_list[i], labels_list[i]):
            y1, x1, y2, x2 = box
            
            # Convert normalized coords to pixel coords
            x1_px, y1_px = x1 * img_w, y1 * img_h
            x2_px, y2_px = x2 * img_w, y2 * img_h
            width_px = x2_px - x1_px
            height_px = y2_px - y1_px
            
            # Draw rectangle
            rect = Rectangle((x1_px, y1_px), width_px, height_px,
                           linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            
            # Add label
            ax.text(x1_px, y1_px-3, f'Building', 
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='red', alpha=0.7),
                   fontsize=8, color='white', weight='bold')
        
        ax.set_title(f'Image {i+1}: {len(boxes_list[i])} buildings', fontweight='bold')
        ax.axis('off')
    
    plt.suptitle(title, fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

# Visualize training samples
visualize_annotated_images(
    train_images, train_boxes, train_labels, 
    n_samples=6, 
    title="Training Set Samples (with Ground Truth Annotations)"
)

### Dataset Statistics

In [ ]:
# Calculate statistics
def calculate_dataset_stats(boxes_list):
    """Calculate bounding box statistics"""
    all_widths = []
    all_heights = []
    all_areas = []
    
    for boxes in boxes_list:
        for box in boxes:
            y1, x1, y2, x2 = box
            width = x2 - x1
            height = y2 - y1
            area = width * height
            
            all_widths.append(width)
            all_heights.append(height)
            all_areas.append(area)
    
    return all_widths, all_heights, all_areas

train_widths, train_heights, train_areas = calculate_dataset_stats(train_boxes)

print("\nTraining Set Statistics:")
print(f"  Total buildings: {len(train_widths)}")
print(f"  Buildings per image: {len(train_widths)/len(train_images):.1f} (avg)")
print(f"\n  Bounding Box Width (normalized):")
print(f"    Mean: {np.mean(train_widths):.3f}")
print(f"    Std:  {np.std(train_widths):.3f}")
print(f"    Range: [{np.min(train_widths):.3f}, {np.max(train_widths):.3f}]")
print(f"\n  Bounding Box Height (normalized):")
print(f"    Mean: {np.mean(train_heights):.3f}")
print(f"    Std:  {np.std(train_heights):.3f}")
print(f"    Range: [{np.min(train_heights):.3f}, {np.max(train_heights):.3f}]")
print(f"\n  Bounding Box Area (normalized):")
print(f"    Mean: {np.mean(train_areas):.4f}")
print(f"    Std:  {np.std(train_areas):.4f}")

### Visualize Distribution

In [ ]:
# Plot distributions
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Width distribution
axes[0].hist(train_widths, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(np.mean(train_widths), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(train_widths):.3f}')
axes[0].set_xlabel('Box Width (normalized)', fontweight='bold')
axes[0].set_ylabel('Count', fontweight='bold')
axes[0].set_title('Bounding Box Width Distribution', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Height distribution
axes[1].hist(train_heights, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1].axvline(np.mean(train_heights), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(train_heights):.3f}')
axes[1].set_xlabel('Box Height (normalized)', fontweight='bold')
axes[1].set_ylabel('Count', fontweight='bold')
axes[1].set_title('Bounding Box Height Distribution', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Area distribution
axes[2].hist(train_areas, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[2].axvline(np.mean(train_areas), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(train_areas):.4f}')
axes[2].set_xlabel('Box Area (normalized)', fontweight='bold')
axes[2].set_ylabel('Count', fontweight='bold')
axes[2].set_title('Bounding Box Area Distribution', fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Dataset statistics look good!")
print("  Wide range of building sizes (small to large)")
print("  Realistic distribution for urban scenes")

---

# Step 3: Data Format & Annotation Understanding (15 minutes)

Understanding data formats is critical for object detection. Let's explore COCO format and conversions.

## Common Annotation Formats

| Format | Box Representation | Normalization | Used By |
|--------|-------------------|---------------|----------|
| **COCO** | [x, y, width, height] | Pixel coords | TensorFlow Object Detection API |
| **Pascal VOC** | [xmin, ymin, xmax, ymax] | Pixel coords | PyTorch, many tools |
| **YOLO** | [x_center, y_center, width, height] | Normalized [0,1] | YOLO models |
| **TF Hub** | [y1, x1, y2, x2] | Normalized [0,1] | TensorFlow Hub models |

Our data uses **TF Hub format**: `[y1, x1, y2, x2]` normalized to [0, 1]

In [ ]:
# Format conversion functions
def tfhub_to_coco(box, img_h, img_w):
    """Convert TF Hub format to COCO format"""
    y1, x1, y2, x2 = box
    x = x1 * img_w
    y = y1 * img_h
    width = (x2 - x1) * img_w
    height = (y2 - y1) * img_h
    return [x, y, width, height]

def tfhub_to_pascal_voc(box, img_h, img_w):
    """Convert TF Hub format to Pascal VOC format"""
    y1, x1, y2, x2 = box
    xmin = x1 * img_w
    ymin = y1 * img_h
    xmax = x2 * img_w
    ymax = y2 * img_h
    return [xmin, ymin, xmax, ymax]

def tfhub_to_yolo(box, img_h, img_w):
    """Convert TF Hub format to YOLO format"""
    y1, x1, y2, x2 = box
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2
    width = x2 - x1
    height = y2 - y1
    return [x_center, y_center, width, height]

# Test conversions
sample_box = train_boxes[0][0]  # First box of first image
img_h, img_w = 320, 320

print("Format Conversion Example:")
print(f"\n  TF Hub (normalized):  {sample_box}")
print(f"  COCO (pixels):        {tfhub_to_coco(sample_box, img_h, img_w)}")
print(f"  Pascal VOC (pixels):  {tfhub_to_pascal_voc(sample_box, img_h, img_w)}")
print(f"  YOLO (normalized):    {tfhub_to_yolo(sample_box, img_h, img_w)}")

---

# Step 4: Load Pre-trained Models (15 minutes)

Transfer learning uses pre-trained models as feature extractors. Let's load multiple architectures from TensorFlow Hub.

## Available Pre-trained Models

| Model | Speed | Accuracy | Best For |
|-------|-------|----------|----------|
| **SSD MobileNet** | Fast | Moderate | Real-time, mobile |
| **EfficientDet D0** | Moderate | Good | Balanced |
| **Faster R-CNN ResNet** | Slow | High | Accuracy-critical |

We'll use **SSD MobileNet V2** for this lab (fast, good for learning).

In [ ]:
print("Loading pre-trained SSD MobileNet V2 from TensorFlow Hub...")
print("(This may take 1-2 minutes on first run)\n")

# Load model from TF Hub
model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"
detector = hub.load(model_url)

print("✅ Pre-trained model loaded successfully!")
print("\nModel Details:")
print("  Architecture: SSD MobileNet V2 with FPN-Lite")
print("  Input size: 320x320 pixels")
print("  Pre-trained on: COCO dataset (80 object classes)")
print("  Use case: Transfer learning for building detection")

### Test Model Inference

Let's run the pre-trained model on one of our images to understand the output format:

In [ ]:
def run_detection(model, image):
    """
    Run object detection on a single image
    
    Args:
        model: Pre-trained TF Hub detector
        image: RGB image (H, W, 3) normalized to [0, 1]
    
    Returns:
        detections: Dictionary with detection outputs
    """
    # Convert to tensor and add batch dimension
    input_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
    input_tensor = input_tensor[tf.newaxis, ...]  # (1, H, W, 3)
    
    # Run inference
    detections = model(input_tensor)
    
    # Convert outputs to numpy
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    return detections

# Test on first training image
test_img = train_images[0]
detections = run_detection(detector, test_img)

print("\nDetection Output Format:")
print(f"  Keys: {list(detections.keys())}")
print(f"\n  Number of detections: {detections['num_detections']}")
print(f"  Detection boxes shape: {detections['detection_boxes'].shape}")
print(f"  Detection scores shape: {detections['detection_scores'].shape}")
print(f"  Detection classes shape: {detections['detection_classes'].shape}")

print(f"\n  Sample detection:")
print(f"    Box (normalized): {detections['detection_boxes'][0]}")
print(f"    Score: {detections['detection_scores'][0]:.3f}")
print(f"    Class: {int(detections['detection_classes'][0])}")

::: {.callout-note}
## Understanding Model Outputs

**detection_boxes**: Bounding box coordinates [y1, x1, y2, x2] normalized to [0, 1]  
**detection_scores**: Confidence scores [0, 1] - higher is better  
**detection_classes**: Class IDs from COCO dataset (1-80)  
**num_detections**: Total number of detections (before filtering)

**Note:** The model outputs many detections (typically 100) with varying confidence. We'll use **Non-Maximum Suppression (NMS)** to filter overlapping boxes.
:::

---

# Step 5: Architecture Comparison (20 minutes)

Let's compare detection architectures to understand speed vs accuracy trade-offs.

## Load Additional Models

In [ ]:
# Model URLs from TensorFlow Hub
models_info = {
    'SSD MobileNet V2': {
        'url': 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
        'input_size': 320,
        'description': 'Fast single-stage detector, good for real-time'
    },
    'EfficientDet D0': {
        'url': 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
        'input_size': 512,
        'description': 'Balanced speed and accuracy, compound scaling'
    }
}

print("Available Detection Models:\n")
for name, info in models_info.items():
    print(f"  {name}:")
    print(f"    Input size: {info['input_size']}x{info['input_size']}")
    print(f"    Description: {info['description']}")
    print()

print("\n💡 For this lab, we'll use SSD MobileNet V2 (already loaded)")
print("   To use EfficientDet, replace the model_url above and reload")

### Benchmark Inference Speed

In [ ]:
import time

def benchmark_model(model, images, n_runs=10):
    """
    Benchmark model inference speed
    """
    print(f"Benchmarking inference speed ({n_runs} runs)...")
    
    times = []
    
    # Warmup
    _ = run_detection(model, images[0])
    
    # Timed runs
    for i in range(n_runs):
        start = time.time()
        _ = run_detection(model, images[i])
        elapsed = time.time() - start
        times.append(elapsed)
    
    mean_time = np.mean(times)
    std_time = np.std(times)
    fps = 1.0 / mean_time
    
    print(f"\n  Inference time: {mean_time*1000:.1f} ± {std_time*1000:.1f} ms")
    print(f"  FPS: {fps:.1f}")
    print(f"  Throughput: {fps*60:.0f} images/minute")
    
    return mean_time, fps

# Benchmark SSD MobileNet
ssd_time, ssd_fps = benchmark_model(detector, test_images, n_runs=10)

print("\n✓ Benchmark complete!")
if ssd_fps > 20:
    print("  Performance: Suitable for near-real-time processing")
elif ssd_fps > 5:
    print("  Performance: Good for batch processing")
else:
    print("  Performance: Suitable for offline analysis")

---

# Step 6: Non-Maximum Suppression (NMS) (20 minutes)

Object detectors output many overlapping boxes. **NMS** filters duplicates to keep only the best detection per object.

## Understanding NMS

**Problem:** Multiple boxes detect the same building  
**Solution:** Keep box with highest confidence, suppress overlapping boxes  
**Method:** Calculate IoU between boxes, remove boxes with IoU > threshold

### Implement IoU Calculation

In [ ]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two boxes
    
    Args:
        box1, box2: Boxes in format [y1, x1, y2, x2] (normalized or pixel)
    
    Returns:
        iou: IoU score [0, 1]
    """
    # Calculate intersection coordinates
    y1_int = max(box1[0], box2[0])
    x1_int = max(box1[1], box2[1])
    y2_int = min(box1[2], box2[2])
    x2_int = min(box1[3], box2[3])
    
    # Check if boxes overlap
    if y2_int <= y1_int or x2_int <= x1_int:
        return 0.0
    
    # Calculate intersection area
    intersection = (y2_int - y1_int) * (x2_int - x1_int)
    
    # Calculate areas of both boxes
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Calculate union area
    union = area1 + area2 - intersection
    
    # Calculate IoU
    iou = intersection / union if union > 0 else 0.0
    
    return iou

# Test IoU calculation
print("IoU Calculation Examples:\n")

# Perfect overlap
box_a = [0.1, 0.1, 0.3, 0.3]
box_b = [0.1, 0.1, 0.3, 0.3]
iou = calculate_iou(box_a, box_b)
print(f"  Perfect overlap: IoU = {iou:.3f}")

# Partial overlap
box_c = [0.2, 0.2, 0.4, 0.4]
iou = calculate_iou(box_a, box_c)
print(f"  Partial overlap: IoU = {iou:.3f}")

# No overlap
box_d = [0.5, 0.5, 0.7, 0.7]
iou = calculate_iou(box_a, box_d)
print(f"  No overlap:      IoU = {iou:.3f}")

### Visualize IoU

In [ ]:
def visualize_iou(box1, box2, img_size=320):
    """
    Visualize two boxes and their IoU
    """
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    
    # Create blank image
    ax.set_xlim(0, img_size)
    ax.set_ylim(img_size, 0)  # Inverted Y-axis
    
    # Convert normalized to pixels
    def to_pixels(box):
        return [c * img_size for c in box]
    
    box1_px = to_pixels(box1)
    box2_px = to_pixels(box2)
    
    # Draw boxes
    rect1 = Rectangle((box1_px[1], box1_px[0]), 
                      box1_px[3]-box1_px[1], box1_px[2]-box1_px[0],
                      linewidth=3, edgecolor='blue', facecolor='blue', alpha=0.3)
    rect2 = Rectangle((box2_px[1], box2_px[0]), 
                      box2_px[3]-box2_px[1], box2_px[2]-box2_px[0],
                      linewidth=3, edgecolor='red', facecolor='red', alpha=0.3)
    
    ax.add_patch(rect1)
    ax.add_patch(rect2)
    
    # Calculate and display IoU
    iou = calculate_iou(box1, box2)
    ax.set_title(f'IoU = {iou:.3f}', fontsize=16, fontweight='bold')
    
    ax.set_xlabel('X (pixels)', fontweight='bold')
    ax.set_ylabel('Y (pixels)', fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='blue', alpha=0.3, edgecolor='blue', label='Box 1'),
                      Patch(facecolor='red', alpha=0.3, edgecolor='red', label='Box 2')]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Visualize different IoU scenarios
print("Visualizing IoU Scenarios:\n")

# High overlap (IoU ~ 0.5)
visualize_iou([0.2, 0.2, 0.5, 0.5], [0.3, 0.3, 0.6, 0.6])

### Implement NMS Algorithm

In [ ]:
def non_max_suppression(boxes, scores, iou_threshold=0.5, score_threshold=0.3):
    """
    Apply Non-Maximum Suppression to remove duplicate detections
    
    Args:
        boxes: Array of bounding boxes [N, 4] in format [y1, x1, y2, x2]
        scores: Array of confidence scores [N]
        iou_threshold: IoU threshold for suppression (default 0.5)
        score_threshold: Minimum score to keep detection (default 0.3)
    
    Returns:
        keep_indices: Indices of boxes to keep
    """
    # Filter by score threshold
    score_mask = scores >= score_threshold
    boxes = boxes[score_mask]
    scores = scores[score_mask]
    
    if len(boxes) == 0:
        return np.array([], dtype=np.int32)
    
    # Sort by scores (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    keep_indices = []
    
    while len(sorted_indices) > 0:
        # Take box with highest score
        current_idx = sorted_indices[0]
        keep_indices.append(current_idx)
        
        if len(sorted_indices) == 1:
            break
        
        # Calculate IoU with remaining boxes
        current_box = boxes[current_idx]
        remaining_boxes = boxes[sorted_indices[1:]]
        
        ious = np.array([calculate_iou(current_box, box) for box in remaining_boxes])
        
        # Keep boxes with IoU below threshold
        keep_mask = ious < iou_threshold
        sorted_indices = sorted_indices[1:][keep_mask]
    
    return np.array(keep_indices, dtype=np.int32)

print("✓ NMS function implemented")
print("\nNMS Parameters:")
print("  iou_threshold: Remove boxes with IoU > 0.5 (default)")
print("  score_threshold: Keep boxes with score > 0.3 (default)")
print("\nThese can be tuned based on your application needs")

### Test NMS on Model Predictions

In [ ]:
# Run detection on test image
test_img = test_images[0]
detections = run_detection(detector, test_img)

# Extract boxes and scores
pred_boxes = detections['detection_boxes']
pred_scores = detections['detection_scores']

print(f"Before NMS: {len(pred_boxes)} detections")
print(f"  Score range: [{pred_scores.min():.3f}, {pred_scores.max():.3f}]")

# Apply NMS
keep_indices = non_max_suppression(
    pred_boxes, 
    pred_scores, 
    iou_threshold=0.5, 
    score_threshold=0.3
)

filtered_boxes = pred_boxes[keep_indices]
filtered_scores = pred_scores[keep_indices]

print(f"\nAfter NMS: {len(filtered_boxes)} detections")
print(f"  Score range: [{filtered_scores.min():.3f}, {filtered_scores.max():.3f}]")
print(f"\n  Reduction: {len(pred_boxes) - len(filtered_boxes)} boxes removed ({(1-len(filtered_boxes)/len(pred_boxes))*100:.1f}%)")

### Visualize NMS Effect

In [ ]:
def visualize_nms_comparison(image, boxes_before, scores_before, boxes_after, scores_after, threshold=0.3):
    """
    Visualize detections before and after NMS
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    # Before NMS
    ax1.imshow(image)
    count_before = 0
    for box, score in zip(boxes_before, scores_before):
        if score > threshold:
            y1, x1, y2, x2 = box
            h, w = 320, 320
            rect = Rectangle((x1*w, y1*h), (x2-x1)*w, (y2-y1)*h,
                           linewidth=2, edgecolor='yellow', facecolor='none')
            ax1.add_patch(rect)
            count_before += 1
    ax1.set_title(f'Before NMS: {count_before} detections', fontsize=14, fontweight='bold')
    ax1.axis('off')
    
    # After NMS
    ax2.imshow(image)
    for box, score in zip(boxes_after, scores_after):
        y1, x1, y2, x2 = box
        h, w = 320, 320
        rect = Rectangle((x1*w, y1*h), (x2-x1)*w, (y2-y1)*h,
                       linewidth=2, edgecolor='lime', facecolor='none')
        ax2.add_patch(rect)
        ax2.text(x1*w, y1*h-5, f'{score:.2f}', 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lime', alpha=0.7),
                fontsize=10, color='black', weight='bold')
    ax2.set_title(f'After NMS: {len(boxes_after)} detections', fontsize=14, fontweight='bold')
    ax2.axis('off')
    
    plt.suptitle('Non-Maximum Suppression Effect', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize NMS effect
visualize_nms_comparison(
    test_img,
    pred_boxes, pred_scores,
    filtered_boxes, filtered_scores,
    threshold=0.3
)

print("\n✓ NMS successfully removed overlapping detections!")
print("  Yellow boxes (before): Many overlaps")
print("  Green boxes (after): Clean, single detection per object")

---

# Step 7: Evaluation with mAP (25 minutes)

**mAP (mean Average Precision)** is the standard metric for object detection. It measures both localization accuracy (IoU) and classification confidence.

## Understanding mAP

**Components:**
1. **IoU Threshold:** Prediction is "correct" if IoU with ground truth > threshold (typically 0.5)
2. **Precision-Recall Curve:** How many detected are correct (precision) vs. how many actual objects found (recall)
3. **Average Precision (AP):** Area under precision-recall curve
4. **mAP:** Mean AP across all classes or IoU thresholds

**Common mAP Variants:**
- **mAP@0.5:** IoU threshold = 0.5 (PASCAL VOC standard)
- **mAP@0.75:** IoU threshold = 0.75 (stricter)
- **mAP@[0.5:0.95]:** Average over IoU thresholds from 0.5 to 0.95 (COCO standard)

### Implement mAP Calculation

In [ ]:
def calculate_ap(gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
    """
    Calculate Average Precision at a specific IoU threshold
    
    Args:
        gt_boxes: Ground truth boxes [N, 4]
        pred_boxes: Predicted boxes [M, 4]
        pred_scores: Prediction scores [M]
        iou_threshold: IoU threshold for correct detection
    
    Returns:
        ap: Average Precision
        precision: Precision values
        recall: Recall values
    """
    if len(pred_boxes) == 0:
        return 0.0, np.array([]), np.array([])
    
    # Sort predictions by confidence (descending)
    sorted_indices = np.argsort(pred_scores)[::-1]
    pred_boxes = pred_boxes[sorted_indices]
    pred_scores = pred_scores[sorted_indices]
    
    # Track which ground truth boxes have been matched
    gt_matched = np.zeros(len(gt_boxes), dtype=bool)
    
    # Track true positives and false positives
    tp = np.zeros(len(pred_boxes))
    fp = np.zeros(len(pred_boxes))
    
    for i, pred_box in enumerate(pred_boxes):
        # Find best matching ground truth box
        best_iou = 0
        best_gt_idx = -1
        
        for j, gt_box in enumerate(gt_boxes):
            if gt_matched[j]:
                continue
            
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = j
        
        # Check if detection is correct
        if best_iou >= iou_threshold and best_gt_idx >= 0:
            tp[i] = 1
            gt_matched[best_gt_idx] = True
        else:
            fp[i] = 1
    
    # Calculate cumulative TP and FP
    tp_cumsum = np.cumsum(tp)
    fp_cumsum = np.cumsum(fp)
    
    # Calculate precision and recall
    precision = tp_cumsum / (tp_cumsum + fp_cumsum + 1e-8)
    recall = tp_cumsum / (len(gt_boxes) + 1e-8)
    
    # Calculate AP (area under precision-recall curve)
    # Use 11-point interpolation (PASCAL VOC style)
    ap = 0
    for t in np.arange(0, 1.1, 0.1):
        if np.sum(recall >= t) == 0:
            p = 0
        else:
            p = np.max(precision[recall >= t])
        ap += p / 11
    
    return ap, precision, recall

print("✓ mAP calculation functions implemented")

### Calculate mAP on Test Set

In [ ]:
def evaluate_map(model, images, gt_boxes_list, iou_thresholds=[0.5, 0.75]):
    """
    Evaluate mAP on a dataset
    """
    print(f"Evaluating mAP on {len(images)} images...\n")
    
    ap_per_threshold = {}
    
    for iou_thresh in iou_thresholds:
        aps = []
        
        for i, (image, gt_boxes) in enumerate(zip(images, gt_boxes_list)):
            # Run detection
            detections = run_detection(model, image)
            pred_boxes = detections['detection_boxes']
            pred_scores = detections['detection_scores']
            
            # Apply NMS
            keep_indices = non_max_suppression(
                pred_boxes, pred_scores, 
                iou_threshold=0.5, 
                score_threshold=0.3
            )
            pred_boxes = pred_boxes[keep_indices]
            pred_scores = pred_scores[keep_indices]
            
            # Calculate AP
            ap, _, _ = calculate_ap(gt_boxes, pred_boxes, pred_scores, iou_threshold=iou_thresh)
            aps.append(ap)
            
            if (i+1) % 5 == 0:
                print(f"  Processed {i+1}/{len(images)} images (IoU={iou_thresh})")
        
        map_value = np.mean(aps)
        ap_per_threshold[iou_thresh] = map_value
        print(f"\n  mAP@{iou_thresh}: {map_value:.3f}")
    
    return ap_per_threshold

# Evaluate on test set
map_results = evaluate_map(detector, test_images, test_boxes, iou_thresholds=[0.5, 0.75])

print("\n" + "="*50)
print("TEST SET RESULTS")
print("="*50)
for thresh, map_val in map_results.items():
    print(f"mAP@{thresh}: {map_val:.3f}")
print("="*50)

### Visualize Precision-Recall Curve

In [ ]:
# Calculate precision-recall for one image
sample_img = test_images[0]
sample_gt = test_boxes[0]

detections = run_detection(detector, sample_img)
pred_boxes = detections['detection_boxes']
pred_scores = detections['detection_scores']

# Apply NMS
keep_indices = non_max_suppression(pred_boxes, pred_scores, iou_threshold=0.5, score_threshold=0.1)
pred_boxes = pred_boxes[keep_indices]
pred_scores = pred_scores[keep_indices]

# Calculate AP and get precision-recall values
ap_05, precision, recall = calculate_ap(sample_gt, pred_boxes, pred_scores, iou_threshold=0.5)

# Plot precision-recall curve
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, 'b-', linewidth=2, label=f'AP@0.5 = {ap_05:.3f}')
plt.fill_between(recall, precision, alpha=0.2)
plt.xlabel('Recall', fontsize=12, fontweight='bold')
plt.ylabel('Precision', fontsize=12, fontweight='bold')
plt.title('Precision-Recall Curve (Sample Image)', fontsize=14, fontweight='bold')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.grid(alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"\n✓ Average Precision: {ap_05:.3f}")
print(f"  This represents the area under the precision-recall curve")

::: {.callout-note}
## Interpreting mAP Results

**mAP Values:**
- **> 0.80:** Excellent performance
- **0.60 - 0.80:** Good performance
- **0.40 - 0.60:** Moderate performance
- **< 0.40:** Poor performance

**For building detection:**
- mAP@0.5 > 0.70 is typically acceptable
- mAP@0.75 > 0.50 indicates good localization accuracy

**Note:** Pre-trained model without fine-tuning may show lower performance. Fine-tuning on real Philippine data would significantly improve results.
:::

---

# Step 8: Advanced Visualization (15 minutes)

Let's create comprehensive visualizations to analyze model performance.

### Confidence Score Distribution

In [ ]:
# Collect all prediction scores from test set
all_scores = []
correct_scores = []
incorrect_scores = []

for image, gt_boxes in zip(test_images, test_boxes):
    detections = run_detection(detector, image)
    pred_boxes = detections['detection_boxes']
    pred_scores = detections['detection_scores']
    
    # Apply NMS
    keep_indices = non_max_suppression(pred_boxes, pred_scores, iou_threshold=0.5, score_threshold=0.3)
    pred_boxes = pred_boxes[keep_indices]
    pred_scores = pred_scores[keep_indices]
    
    all_scores.extend(pred_scores)
    
    # Check which predictions are correct
    for pred_box, pred_score in zip(pred_boxes, pred_scores):
        # Find best matching GT box
        best_iou = 0
        for gt_box in gt_boxes:
            iou = calculate_iou(pred_box, gt_box)
            best_iou = max(best_iou, iou)
        
        if best_iou >= 0.5:
            correct_scores.append(pred_score)
        else:
            incorrect_scores.append(pred_score)

# Plot score distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# All scores
axes[0].hist(all_scores, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].axvline(0.5, color='red', linestyle='--', linewidth=2, label='Common threshold')
axes[0].set_xlabel('Confidence Score', fontweight='bold')
axes[0].set_ylabel('Count', fontweight='bold')
axes[0].set_title('All Prediction Scores', fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Correct vs incorrect
axes[1].hist(correct_scores, bins=20, alpha=0.7, color='green', label='Correct (IoU≥0.5)', edgecolor='black')
axes[1].hist(incorrect_scores, bins=20, alpha=0.7, color='red', label='Incorrect (IoU<0.5)', edgecolor='black')
axes[1].axvline(0.5, color='black', linestyle='--', linewidth=2, label='Threshold')
axes[1].set_xlabel('Confidence Score', fontweight='bold')
axes[1].set_ylabel('Count', fontweight='bold')
axes[1].set_title('Score Distribution by Correctness', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nScore Statistics:")
print(f"  Total predictions: {len(all_scores)}")
print(f"  Correct (IoU≥0.5): {len(correct_scores)} ({len(correct_scores)/len(all_scores)*100:.1f}%)")
print(f"  Incorrect (IoU<0.5): {len(incorrect_scores)} ({len(incorrect_scores)/len(all_scores)*100:.1f}%)")
print(f"\n  Mean score (correct): {np.mean(correct_scores):.3f}")
print(f"  Mean score (incorrect): {np.mean(incorrect_scores):.3f}")

### Error Analysis: Visualize Failures

In [ ]:
def visualize_detections_with_analysis(image, gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
    """
    Visualize detections with error analysis
    Green = True Positive (correct detection)
    Red = False Positive (incorrect detection)
    Yellow = False Negative (missed ground truth)
    """
    plt.figure(figsize=(12, 10))
    plt.imshow(image)
    
    # Track which GT boxes are matched
    gt_matched = np.zeros(len(gt_boxes), dtype=bool)
    
    # Process predictions
    tp_count = 0
    fp_count = 0
    
    for pred_box, pred_score in zip(pred_boxes, pred_scores):
        y1, x1, y2, x2 = pred_box
        h, w = 320, 320
        
        # Find best matching GT
        best_iou = 0
        best_gt_idx = -1
        for i, gt_box in enumerate(gt_boxes):
            if not gt_matched[i]:
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = i
        
        # Color based on correctness
        if best_iou >= iou_threshold and best_gt_idx >= 0:
            color = 'lime'
            label = f'TP: {pred_score:.2f}'
            gt_matched[best_gt_idx] = True
            tp_count += 1
        else:
            color = 'red'
            label = f'FP: {pred_score:.2f}'
            fp_count += 1
        
        rect = Rectangle((x1*w, y1*h), (x2-x1)*w, (y2-y1)*h,
                       linewidth=2, edgecolor=color, facecolor='none')
        plt.gca().add_patch(rect)
        plt.text(x1*w, y1*h-5, label,
                bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.7),
                fontsize=9, color='black', weight='bold')
    
    # Draw unmatched GT boxes (False Negatives)
    fn_count = 0
    for i, gt_box in enumerate(gt_boxes):
        if not gt_matched[i]:
            y1, x1, y2, x2 = gt_box
            h, w = 320, 320
            rect = Rectangle((x1*w, y1*h), (x2-x1)*w, (y2-y1)*h,
                           linewidth=3, edgecolor='yellow', facecolor='none', linestyle='--')
            plt.gca().add_patch(rect)
            plt.text(x1*w, y1*h-5, 'FN (Missed)',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                    fontsize=9, color='black', weight='bold')
            fn_count += 1
    
    plt.title(f'Detection Analysis | TP: {tp_count} | FP: {fp_count} | FN: {fn_count}',
             fontsize=14, fontweight='bold')
    plt.axis('off')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='lime', label='True Positive (Correct)'),
        Patch(facecolor='red', label='False Positive (Wrong)'),
        Patch(facecolor='yellow', label='False Negative (Missed)')
    ]
    plt.legend(handles=legend_elements, loc='upper right', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    return tp_count, fp_count, fn_count

# Analyze several test images
print("Error Analysis on Test Images:\n")

for i in range(3):
    img = test_images[i]
    gt = test_boxes[i]
    
    # Run detection
    detections = run_detection(detector, img)
    pred_boxes = detections['detection_boxes']
    pred_scores = detections['detection_scores']
    
    # Apply NMS
    keep_indices = non_max_suppression(pred_boxes, pred_scores, iou_threshold=0.5, score_threshold=0.4)
    pred_boxes = pred_boxes[keep_indices]
    pred_scores = pred_scores[keep_indices]
    
    print(f"Image {i+1}:")
    tp, fp, fn = visualize_detections_with_analysis(img, gt, pred_boxes, pred_scores, iou_threshold=0.5)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    print(f"  Precision: {precision:.3f} | Recall: {recall:.3f}\n")

---

# Step 9: Export & Deployment (15 minutes)

For operational use, we need to export models for deployment.

## Save Detection Results

In [ ]:
import json

# Create output directory
os.makedirs('/content/detection_outputs', exist_ok=True)

def export_detections_to_json(images, boxes_list, scores_list, output_file):
    """
    Export detections to JSON format (COCO-like)
    """
    results = []
    
    for img_id, (image, boxes, scores) in enumerate(zip(images, boxes_list, scores_list)):
        img_h, img_w = 320, 320
        
        for box, score in zip(boxes, scores):
            y1, x1, y2, x2 = box
            
            # Convert to COCO format [x, y, width, height] in pixels
            x_px = x1 * img_w
            y_px = y1 * img_h
            w_px = (x2 - x1) * img_w
            h_px = (y2 - y1) * img_h
            
            results.append({
                'image_id': img_id,
                'category_id': 1,  # Building class
                'bbox': [float(x_px), float(y_px), float(w_px), float(h_px)],
                'score': float(score)
            })
    
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"✓ Exported {len(results)} detections to {output_file}")

# Run detection on test set and export
test_pred_boxes_list = []
test_pred_scores_list = []

for image in test_images:
    detections = run_detection(detector, image)
    pred_boxes = detections['detection_boxes']
    pred_scores = detections['detection_scores']
    
    # Apply NMS
    keep_indices = non_max_suppression(pred_boxes, pred_scores, iou_threshold=0.5, score_threshold=0.3)
    pred_boxes = pred_boxes[keep_indices]
    pred_scores = pred_scores[keep_indices]
    
    test_pred_boxes_list.append(pred_boxes)
    test_pred_scores_list.append(pred_scores)

# Export to JSON
export_detections_to_json(
    test_images, 
    test_pred_boxes_list, 
    test_pred_scores_list,
    '/content/detection_outputs/test_predictions.json'
)

### Export to GeoJSON (for GIS Integration)

In [ ]:
def export_to_geojson(boxes_list, scores_list, output_file, origin_lon=121.0, origin_lat=14.6, pixel_size=10):
    """
    Export detections to GeoJSON format for GIS integration
    
    Args:
        boxes_list: List of detection boxes
        scores_list: List of confidence scores
        output_file: Output GeoJSON file path
        origin_lon: Longitude of image origin (upper-left)
        origin_lat: Latitude of image origin (upper-left)
        pixel_size: Pixel size in meters (default 10m for Sentinel-2)
    """
    # Approximate degrees per meter at latitude
    meters_per_degree_lat = 111320.0
    meters_per_degree_lon = 111320.0 * np.cos(np.radians(origin_lat))
    
    features = []
    
    for img_id, (boxes, scores) in enumerate(zip(boxes_list, scores_list)):
        for box, score in zip(boxes, scores):
            y1, x1, y2, x2 = box
            
            # Convert pixel coords to geographic coords
            img_h, img_w = 320, 320
            
            # Top-left corner in pixels
            x1_px = x1 * img_w
            y1_px = y1 * img_h
            x2_px = x2 * img_w
            y2_px = y2 * img_h
            
            # Convert to meters
            x1_m = x1_px * pixel_size
            y1_m = y1_px * pixel_size
            x2_m = x2_px * pixel_size
            y2_m = y2_px * pixel_size
            
            # Convert to geographic coordinates
            lon1 = origin_lon + (x1_m / meters_per_degree_lon)
            lat1 = origin_lat - (y1_m / meters_per_degree_lat)
            lon2 = origin_lon + (x2_m / meters_per_degree_lon)
            lat2 = origin_lat - (y2_m / meters_per_degree_lat)
            
            # Create polygon (bounding box)
            coordinates = [[
                [lon1, lat1],
                [lon2, lat1],
                [lon2, lat2],
                [lon1, lat2],
                [lon1, lat1]
            ]]
            
            feature = {
                'type': 'Feature',
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': coordinates
                },
                'properties': {
                    'image_id': img_id,
                    'class': 'building',
                    'confidence': float(score),
                    'area_m2': (x2_m - x1_m) * (y2_m - y1_m)
                }
            }
            features.append(feature)
    
    geojson = {
        'type': 'FeatureCollection',
        'crs': {
            'type': 'name',
            'properties': {'name': 'EPSG:4326'}
        },
        'features': features
    }
    
    with open(output_file, 'w') as f:
        json.dump(geojson, f, indent=2)
    
    print(f"✓ Exported {len(features)} building polygons to {output_file}")
    print(f"  Ready for QGIS/ArcGIS import")

# Export to GeoJSON
export_to_geojson(
    test_pred_boxes_list,
    test_pred_scores_list,
    '/content/detection_outputs/buildings.geojson',
    origin_lon=121.0,  # Metro Manila approximate
    origin_lat=14.6,
    pixel_size=10  # Sentinel-2 resolution
)

### Create Summary Report

In [ ]:
# Generate summary report
report = {
    'model': 'SSD MobileNet V2',
    'dataset': {
        'train_images': len(train_images),
        'val_images': len(val_images),
        'test_images': len(test_images),
        'total_buildings': sum(len(b) for b in test_boxes)
    },
    'performance': {
        'mAP@0.5': float(map_results.get(0.5, 0)),
        'mAP@0.75': float(map_results.get(0.75, 0)),
        'inference_time_ms': float(ssd_time * 1000),
        'fps': float(ssd_fps)
    },
    'detections': {
        'total_predicted': sum(len(b) for b in test_pred_boxes_list),
        'mean_per_image': float(np.mean([len(b) for b in test_pred_boxes_list])),
        'mean_confidence': float(np.mean([s.mean() for s in test_pred_scores_list if len(s) > 0]))
    },
    'nms_config': {
        'iou_threshold': 0.5,
        'score_threshold': 0.3
    }
}

with open('/content/detection_outputs/summary_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\n📊 Summary Report")
print("="*50)
print(f"Model: {report['model']}")
print(f"\nDataset:")
for key, val in report['dataset'].items():
    print(f"  {key}: {val}")
print(f"\nPerformance:")
for key, val in report['performance'].items():
    print(f"  {key}: {val:.3f}")
print(f"\nDetections:")
for key, val in report['detections'].items():
    print(f"  {key}: {val:.2f}")
print("="*50)

print("\n✓ All outputs saved to /content/detection_outputs/")

---

# Step 10: Troubleshooting & Best Practices (10 minutes)

## Common Issues and Solutions

::: {.callout-warning}
## Issue 1: Low mAP / Poor Detection Performance

**Symptoms:**
- mAP < 0.40
- Many false positives or false negatives
- Model detects wrong objects

**Solutions:**

1. **Fine-tune on domain-specific data:**
   ```python
   # Pre-trained model is trained on COCO (general objects)
   # Need to fine-tune on Philippine building dataset
   # Use TensorFlow Object Detection API for fine-tuning
   ```

2. **Adjust NMS thresholds:**
   ```python
   # Try different IoU thresholds
   keep_indices = non_max_suppression(
       boxes, scores,
       iou_threshold=0.3,  # Lower = keep more boxes
       score_threshold=0.5  # Higher = more confident only
   )
   ```

3. **Collect more training data:**
   - Minimum 500-1000 annotated images
   - Diverse building sizes and types
   - Various lighting and seasons
:::

::: {.callout-warning}
## Issue 2: Slow Inference Speed

**Symptoms:**
- FPS < 5
- Long processing time for large areas

**Solutions:**

1. **Use lighter model:**
   ```python
   # Switch to MobileNet (current) or SSD Lite
   model_url = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"
   ```

2. **Optimize model:**
   ```python
   # Export to TFLite for mobile/edge deployment
   converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
   converter.optimizations = [tf.lite.Optimize.DEFAULT]
   tflite_model = converter.convert()
   ```

3. **Batch processing:**
   ```python
   # Process multiple images at once
   batch_images = tf.stack([img1, img2, img3, img4])
   detections = model(batch_images)
   ```

4. **Use GPU:**
   - Ensure GPU is available in Colab (Runtime → Change runtime type)
   - Check with `tf.config.list_physical_devices('GPU')`
:::

::: {.callout-warning}
## Issue 3: Too Many False Positives

**Symptoms:**
- Model detects buildings where there are none
- Low precision (<0.5)

**Solutions:**

1. **Increase confidence threshold:**
   ```python
   keep_indices = non_max_suppression(
       boxes, scores,
       score_threshold=0.6  # Increase from 0.3
   )
   ```

2. **Add negative examples:**
   - Include images with no buildings in training
   - Annotate "hard negatives" (vegetation, rocks that look like buildings)

3. **Use contextual filtering:**
   ```python
   # Remove detections outside urban areas
   # Use land cover mask to filter unlikely locations
   ```
:::

::: {.callout-warning}
## Issue 4: Missing Small Buildings

**Symptoms:**
- Small buildings not detected
- Low recall for small objects

**Solutions:**

1. **Use higher resolution input:**
   ```python
   # Use 512x512 or 640x640 instead of 320x320
   # Note: Slower inference
   ```

2. **Multi-scale detection:**
   ```python
   # Run detection at multiple scales
   scales = [0.75, 1.0, 1.25]
   all_detections = []
   for scale in scales:
       resized = tf.image.resize(image, [int(320*scale), int(320*scale)])
       detections = model(resized)
       all_detections.append(detections)
   # Combine and apply NMS
   ```

3. **Use architecture with FPN:**
   - Feature Pyramid Networks handle multi-scale better
   - EfficientDet has built-in BiFPN
:::

## Best Practices for Production

### 1. Data Preparation
✅ Use high-quality annotations (double-check bounding boxes)  
✅ Balance dataset across building sizes  
✅ Include seasonal variations (wet/dry season)  
✅ Annotate at least 1000 images for good performance

### 2. Model Selection
✅ **Fast inference needed:** SSD MobileNet  
✅ **Balanced:** EfficientDet D0-D2  
✅ **High accuracy:** Faster R-CNN ResNet

### 3. Hyperparameter Tuning
✅ **NMS IoU threshold:** Start 0.5, tune 0.3-0.7  
✅ **Score threshold:** Start 0.3, tune 0.2-0.6  
✅ **Test on validation set first**

### 4. Evaluation
✅ Report mAP@0.5 and mAP@0.75  
✅ Calculate per-class metrics  
✅ Analyze false positives and false negatives  
✅ Test on diverse geographic regions

### 5. Deployment
✅ Export to TFLite or ONNX for production  
✅ Monitor inference time and memory usage  
✅ Implement batch processing for large areas  
✅ Add geographic post-processing (filter by land cover)

---

# 🎉 Lab Complete!

## What You've Accomplished

### Technical Skills:
✅ **Loaded** pre-trained object detection models from TensorFlow Hub  
✅ **Generated** synthetic urban satellite imagery with annotations  
✅ **Implemented** Non-Maximum Suppression (NMS) from scratch  
✅ **Calculated** mAP (mean Average Precision) for model evaluation  
✅ **Compared** detection architectures (speed vs accuracy)  
✅ **Visualized** detections with comprehensive error analysis  
✅ **Exported** results in multiple formats (JSON, GeoJSON)  
✅ **Applied** transfer learning for Philippine building detection

### Conceptual Understanding:
✅ Transfer learning for object detection  
✅ Bounding box formats (COCO, Pascal VOC, YOLO, TF Hub)  
✅ NMS algorithm and its importance  
✅ mAP metric and precision-recall curves  
✅ IoU calculation and thresholds  
✅ Model architecture trade-offs (SSD vs Faster R-CNN)  
✅ Error types (TP, FP, FN) and their operational implications

### Philippine Urban Monitoring Context:
✅ Building detection for disaster risk reduction  
✅ Informal settlement mapping for urban planning  
✅ Change detection for infrastructure development  
✅ GIS integration for operational use

---

## Key Takeaways

### 1. Transfer Learning Works
Pre-trained models (COCO dataset) provide good starting point for building detection, even without fine-tuning.

### 2. NMS is Essential
Object detectors output many overlapping boxes. NMS filters duplicates to give clean results.

### 3. mAP Tells the Full Story
mAP combines localization accuracy and classification confidence into single metric. Always report mAP@0.5 and mAP@0.75.

### 4. Threshold Tuning Matters
NMS IoU threshold and confidence threshold significantly affect results. Tune on validation set.

### 5. Speed vs Accuracy Trade-off
- **Real-time:** SSD MobileNet
- **Balanced:** EfficientDet
- **Accuracy-critical:** Faster R-CNN

### 6. Domain-Specific Fine-tuning Improves Performance
For production use, fine-tune on Philippine building dataset using TensorFlow Object Detection API.

---

## Next Steps for Production

### 1. Real Data Collection
- Acquire Sentinel-2 imagery for Metro Manila
- Download from [Copernicus Open Access Hub](https://scihub.copernicus.eu/)
- Or use Google Earth Engine

### 2. Annotation
- Use [RoboFlow](https://roboflow.com/) or [CVAT](https://cvat.org/) for bounding box annotation
- Annotate 500-1000 images minimum
- Include diverse building types and sizes

### 3. Fine-tuning
- Use [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)
- Fine-tune SSD or EfficientDet on Philippine data
- Train for 20K-50K steps

### 4. Deployment
- Export to TFLite for edge deployment
- Create batch processing pipeline
- Integrate with GIS workflows
- Deploy to PhilSA/DOST operational systems

### 5. Monitoring
- Track inference time and accuracy
- Collect edge cases for retraining
- Update model quarterly with new data

---

## Resources for Further Learning

### Papers
- [SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)
- [Faster R-CNN](https://arxiv.org/abs/1506.01497)
- [EfficientDet](https://arxiv.org/abs/1911.09070)

### Tutorials
- [TensorFlow Object Detection API Tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/)
- [RoboFlow Object Detection Guide](https://blog.roboflow.com/object-detection/)

### Datasets
- [xView Dataset](http://xviewdataset.org/) - Satellite object detection
- [DOTA Dataset](https://captain-whu.github.io/DOTA/) - Aerial image object detection
- [SpaceNet](https://spacenet.ai/) - Building footprint detection

### Tools
- [TensorFlow Hub](https://tfhub.dev/) - Pre-trained models
- [RoboFlow](https://roboflow.com/) - Dataset management and annotation
- [Weights & Biases](https://wandb.ai/) - Experiment tracking

---

## Discussion Questions

1. **Application Design:**
   - How would you deploy this building detection system for Metro Manila monitoring?
   - What infrastructure and data pipelines are needed?

2. **Model Selection:**
   - When would you choose SSD over Faster R-CNN for Philippine use cases?
   - How do you balance speed vs accuracy requirements?

3. **Evaluation:**
   - Is mAP@0.5 > 0.70 sufficient for disaster risk reduction applications?
   - Should we prioritize precision or recall for informal settlement mapping?

4. **Data Quality:**
   - How many training images are needed for operational accuracy?
   - How do you handle seasonal variations in Philippine imagery?

5. **Operational Challenges:**
   - What happens if the model misses a building in a flood zone (false negative)?
   - How do you validate predictions in remote areas with no ground truth?

6. **Integration:**
   - How would you integrate detections with existing NAMRIA/PhilSA databases?
   - What quality control steps are needed before operational use?

---

## Expected Results Summary

| Metric | Expected Range | Interpretation |
|--------|----------------|----------------|
| **mAP@0.5** | 0.40 - 0.70 | Moderate (pre-trained, no fine-tuning) |
| **mAP@0.75** | 0.25 - 0.50 | Good localization given constraints |
| **Inference Time** | 50-150 ms | Fast enough for batch processing |
| **FPS** | 5-20 | Suitable for operational use |
| **Precision** | 0.50 - 0.80 | Few false alarms |
| **Recall** | 0.60 - 0.85 | Catches most buildings |

::: {.callout-note}
## With Fine-tuning on Philippine Data

**Expected improvements:**
- mAP@0.5: **0.70 - 0.90** (+30-40%)
- mAP@0.75: **0.50 - 0.75** (+100%)
- Precision: **0.75 - 0.92**
- Recall: **0.80 - 0.95**

Fine-tuning on domain-specific data typically doubles performance!
:::

---

## Lab Completion Checklist

Before finishing, ensure you've completed:

- [ ] Generated synthetic urban imagery dataset
- [ ] Loaded pre-trained SSD MobileNet V2 model
- [ ] Implemented and tested IoU calculation
- [ ] Implemented Non-Maximum Suppression
- [ ] Calculated mAP@0.5 and mAP@0.75
- [ ] Visualized precision-recall curves
- [ ] Analyzed detection errors (TP, FP, FN)
- [ ] Exported results to JSON and GeoJSON
- [ ] Generated summary report
- [ ] Understood troubleshooting strategies

::: {.callout-success}
## Congratulations! 🎊

You've completed a full object detection pipeline for satellite imagery analysis!

**What You Built:**
- A working building detection system
- Complete evaluation framework (mAP, precision-recall)
- Production-ready export workflows
- GIS-compatible outputs

**Impact:**
Your skills can now contribute to urban monitoring, disaster risk reduction, and infrastructure planning for Philippine cities.

**Next:** Apply these techniques to Day 4 advanced topics (change detection, time series analysis)
:::

---

::: {.session-nav}
[← Back to Session 3](../sessions/session3.qmd){.btn .btn-outline-secondary}
[Course Summary →](../../index.qmd){.btn .btn-primary}
:::

---

*This hands-on lab is part of the CoPhil 4-Day Advanced Training on AI/ML for Earth Observation, funded by the European Union under the Global Gateway initiative. Materials developed in collaboration with PhilSA, DOST-ASTI, and the European Space Agency.*